In [23]:
import os
import warnings
import xml.etree.ElementTree as ET
import pandas as pd

df = pd.read_csv("name_map_old_names.csv")
df

,old_name
0,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series00
1,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series01
2,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series02
3,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series03
4,20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series04
...,...
1661,20240307_bmcc83_fm_hpf_nhs_cent_tub_dna_60x_015
1662,20240307_bmcc83_fm_hpf_nhs_cent_tub_dna_60x_016
1663,20240307_bmcc83_fm_hpf_nhs_cent_tub_dna_60x_017
1664,20240307_bmcc83_fm_hpf_nhs_cent_tub_dna_60x_018


In [24]:
filenames = df["old_name"].tolist()
filenames

['20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series00',
 '20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series01',
 '20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series02',
 '20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series03',
 '20230616_RCC_6731_pfa_OD_Hoechst-tub-nhs_series04',
 '20230803_rccp22-88_pfa_centr-tub-hoechst_series00',
 '20230803_rccp22-88_pfa_centr-tub-hoechst_series01',
 '20230803_rccp22-88_pfa_centr-tub-hoechst_series02',
 '20230803_rccp22-88_pfa_centr-tub-hoechst_series03',
 '20230803_rccp22-88_pfa_centr-tub-hoechst_series04',
 '20230803_rccp22-88_pfa_centr-tub-hoechst_series05',
 '20230516_rcc2962_pfa_hoechst-nhs-tub_series00',
 '20230516_rcc2962_pfa_hoechst-nhs-tub_series01',
 '20230516_rcc2962_pfa_hoechst-nhs-tub_series02',
 '20230516_rcc2962_pfa_hoechst-nhs-tub_series03',
 '20230516_rcc2962_pfa_hoechst-nhs-tub_series04',
 '20230516_rcc2962_pfa_hoechst-nhs-tub_series05',
 '20230516_rcc2962_pfa_hoechst-nhs-tub_series06',
 '20230516_rcc2962_pfa_hoechst-nhs-tub_series07',
 '2023

In [26]:
dir = "/home/hellgoth/culture-collections/data/single_volumes/images/ome-zarr/"
size_t: list[str] = []
size_c: list[str] = []
size_z: list[str] = []
size_y: list[str] = []
size_x: list[str] = []
xy_mismatch: list[str] = []
physical_size_z: list[str] = []
physical_size_y: list[str] = []
physical_size_x: list[str] = []
physical_unit: list[str] = []
physical_xy_mismatch: list[str] = []
has_lng_lvv_in_name: list[str] = []
for name in filenames:
    series = 0
    name_split = name.split("_")
    # if one zarr contains multiple vols
    if name_split[-1].startswith("series"):
        series = int(name_split[-1].split("series")[-1])
        name = "_".join(name_split[:-1])
    meta = dir + name + ".ome.zarr/OME/METADATA.ome.xml"
    # normal case
    if os.path.exists(meta):
        tree = ET.parse(meta)
        root = tree.getroot()
        imgs = []
        for el in root.iter():
            if el.tag.endswith("Image"):
                imgs.append(el)
        for el in imgs[series].iter():
            if el.tag.endswith("Pixels"):
                break
        size_t.append(el.attrib["SizeT"])
        size_c.append(el.attrib["SizeC"])
        size_z.append(el.attrib["SizeZ"])
        size_y.append(el.attrib["SizeY"])
        size_x.append(el.attrib["SizeX"])
        xy_mismatch.append(
            True if el.attrib["SizeX"] != el.attrib["SizeY"] else False
        )
        pz = el.attrib["PhysicalSizeZ"] if "PhysicalSizeZ" in el.attrib else None
        physical_size_z.append(pz)
        py = el.attrib["PhysicalSizeY"] if "PhysicalSizeY" in el.attrib else None
        physical_size_y.append(py)
        px = el.attrib["PhysicalSizeX"] if "PhysicalSizeX" in el.attrib else None
        physical_size_x.append(px)
        puz = el.attrib["PhysicalSizeZUnit"] if "PhysicalSizeZUnit" in el.attrib else None
        puy = el.attrib["PhysicalSizeYUnit"] if "PhysicalSizeYUnit" in el.attrib else None
        pux = el.attrib["PhysicalSizeXUnit"] if "PhysicalSizeXUnit" in el.attrib else None
        if puy is not None:
            assert puy == pux, f"{name}, {puy}, {pux}"
        if puz is not None:
            assert puz == puy, f"{name}, {puz}, {puy}"
        physical_unit.append(pux)
        physical_xy_mismatch.append(
            True if px != py else False
        )
        has_lng_lvv_in_name.append(
            True if "Lng_LVCC" in imgs[series].attrib["Name"] else False
        )
    # assume case where one vol per channel
    else:
        channels = name_split[-2].split("-")
        names = [
            "_".join(name_split[:-2] + [channel] + name_split[-1:])
            for channel in channels
        ]
        ts: list[str] = []
        # cs: list[str] = []
        zs: list[str] = []
        ys: list[str] = []
        xs: list[str] = []
        pzs: list[str] = []
        pys: list[str] = []
        pxs: list[str] = []
        pus: list[str] = []
        lng_lvvs: list[str] = []
        for n in names:
            meta = dir + n + ".ome.zarr/OME/METADATA.ome.xml"
            tree = ET.parse(meta)
            root = tree.getroot()
            imgs = []
            for el in root.iter():
                if el.tag.endswith("Image"):
                    imgs.append(el)
            for el in imgs[series].iter():
                if el.tag.endswith("Pixels"):
                    break
            ts.append(el.attrib["SizeT"])
            zs.append(el.attrib["SizeZ"])
            ys.append(el.attrib["SizeY"])
            xs.append(el.attrib["SizeX"])
            pz = el.attrib["PhysicalSizeZ"] if "PhysicalSizeZ" in el.attrib else None
            pzs.append(pz)
            py = el.attrib["PhysicalSizeY"] if "PhysicalSizeY" in el.attrib else None
            pys.append(py)
            px = el.attrib["PhysicalSizeX"] if "PhysicalSizeX" in el.attrib else None
            pxs.append(px)
            puz = el.attrib["PhysicalSizeZUnit"] if "PhysicalSizeZUnit" in el.attrib else None
            puy = el.attrib["PhysicalSizeYUnit"] if "PhysicalSizeYUnit" in el.attrib else None
            pux = el.attrib["PhysicalSizeXUnit"] if "PhysicalSizeXUnit" in el.attrib else None
            if puy is not None:
                assert puy == pux, f"{name}, {puy}, {pux}"
            if puz is not None:
                assert puz == puy, f"{name}, {puz}, {puy}"
            pus.append(pux)
            lng_lvvs.append(
                True if "Lng_LVCC" in imgs[series].attrib["Name"] else False
            )
        if not all(x == ts[0] for x in ts):
            warnings.warn(f"{name}, {ts}")
        if not all(x == zs[0] for x in zs):
            warnings.warn(f"{name}, {zs}")
        if not all(x == ys[0] for x in ys):
            warnings.warn(f"{name}, {ys}")
        if not all(x == xs[0] for x in xs):
            warnings.warn(f"{name}, {xs}")
        if not all(x == pzs[0] for x in pzs):
            warnings.warn(f"{name}, {pzs}")
        if not all(x == pys[0] for x in pys):
            warnings.warn(f"{name}, {pys}")
        if not all(x == pxs[0] for x in pxs):
            warnings.warn(f"{name}, {pxs}")
        if not all(x == pus[0] for x in pus):
            warnings.warn(f"{name}, {pus}")
        if not all(x == lng_lvvs[0] for x in lng_lvvs):
            warnings.warn(f"{name}, {lng_lvvs}")
        size_t.append(ts[0])
        size_c.append(len(channels))
        size_z.append(zs[0])
        size_y.append(ys[0])
        size_x.append(xs[0])
        xy_mismatch.append(
            True if xs[0] != ys[0] else False
        )
        physical_size_z.append(pzs[0])
        physical_size_y.append(pys[0])
        physical_size_x.append(pxs[0])
        physical_unit.append(pus[0])
        physical_xy_mismatch.append(
            True if pxs[0] != pys[0] else False
        )
        has_lng_lvv_in_name.append(lng_lvvs[0])

/tmp/ipykernel_223080/3109891919.py:113: UserWarning: 20240213_bmcc_16_pfa_hs_cent-dna-nhs-tub_07, ['133', '19', '133', '133']
  warnings.warn(f"{name}, {zs}")
/tmp/ipykernel_223080/3109891919.py:119: UserWarning: 20240213_bmcc_16_pfa_hs_cent-dna-nhs-tub_07, ['0.3', '0.2999999999999798', '0.3', '0.3']
  warnings.warn(f"{name}, {pzs}")


In [27]:
len(size_t)

1666

In [28]:
df = pd.DataFrame(
    {
        "old_name": filenames,
        "size_t": size_t,
        "size_c": size_c,
        "size_z": size_z,
        "size_y": size_y,
        "size_x": size_x,
        "xy_mismatch": xy_mismatch,
        "physical_size_z": physical_size_z,
        "physical_size_y": physical_size_y,
        "physical_size_x": physical_size_x,
        "physical_unit": physical_unit,
        "physical_xy_mismatch": physical_xy_mismatch,
        "has_lng_lvv_in_name": has_lng_lvv_in_name,
    }
)
df.to_csv("additional_info.csv", sep="\t", index=False)